In [ ]:
!pip install opencv-python

In [ ]:
#import packages that allow for image tranformations and transform to Tensor
from skimage.transform import resize
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
import torch
import os
import imageio
import cv2 as cv

In [ ]:
#loop through the directory and transform to tensor
path = "/Users/danielcastillo/Documents/senior-project/Data/CATS_DOGS/test/CAT"

#declare a new list
arr = []
#to clean up the appending process
from tqdm.notebook import tqdm
tqdm().pandas()
i = 0

for pic in tqdm(os.scandir(path)):
    if i < 100:
    #control the number of images to transform
        if (pic.path.endswith(".jpeg") or pic.path.endswith(".jpg")) and pic.is_file():
            #read the image
            picture = cv.imread(pic.path)
            #print(type(picture))
            #check if None Type
            if(picture is None):
                print(i, "not added to the list")
                continue
            else:
                #print(i, "shape: ",picture.shape)
                #resize the picture
                picture = resize(picture, (10,10), mode='constant', anti_aliasing=True)
                #change the picture to gray scale
                picture = rgb2gray(picture)
                #print("shape: ", picture.shape)
                #transform the picture to a tensor
                Tensor = torch.tensor(picture)
                Tensor = Tensor.flatten()
                #append the tensor to the list
                arr.append(Tensor)
                #print(i, "added to the list")
                #print("--------------------------------")
    i+=1

In [ ]:
len(arr)

In [ ]:
#Create a new duet session
import syft as sy
sy.VERBOSE = False
duet = sy.duet(loopback = True)

In [ ]:
import tensorflow as tf
#change the list of tensors into a single tensor
stacked = torch.stack(arr)
#print(stacked)
#change the tensor data type from Float64 to Float32
stacked = stacked.type(torch.FloatTensor)

In [ ]:
print(stacked.dtype)
print(type(stacked))
print(stacked.shape)

In [ ]:
for index in arr:
    if(index is None):
        print(index)

In [ ]:
#assign a tag to the image
image = stacked.tag("test")
#assign a description
image = stacked.describe("tensor data")

#send the pointer through duet
image_ptr = image.send(duet, searchable=True)

In [ ]:
for index in stacked:
    print(type(index))

In [ ]:
#check all the pointers in the duet store
duet.store.pandas

In [ ]:
#add a hadler to accept any request with the name "loss"
duet.requests.add_handler(
    name="loss",
    action="accept",
    timeout_secs=-1,  # no timeout
    print_local=True  # print the result in your notebook
)

In [ ]:
#add a hadler to accept any request with the name "params"
duet.requests.add_handler(
    name="params",
    action="accept",
    timeout_secs=-1,  # no timeout
    print_local=True  # print the result in your notebook
)

In [ ]:
#add a hadler to accept any request with the name "model_download"
duet.requests.add_handler(
    name="model_download",
    action="accept",
    timeout_secs=-1,  # no timeout
    print_local=True  # print the result in your notebook
)

In [ ]:
for index in arr:
    print(type(index))